In [1]:
import torch
from torch import tensor, nn, optim

### LayerNorm

In [4]:
class LayerNorm(nn.Module):
  def __init__(self, eps=1e-5):
    super().__init__()
    self.eps = eps
    self.mult = nn.parameter(tensor(1.))
    self.add = nn.parameter(tensor(0.))

  def forward(self, x):
    m = x.mean((1,2,3), keepdim=True) #taking mean over CHW dimensions
    v = x.var((1,2,3), keepdim=True)
    x = (x - m)/((v + self.eps).sqrt())
    return self.x+self.mult + self.add

In [5]:
class BatchNorm(nn.Module):
  def __init__(self, nf, mom=0.1, eps=1e-5):
    super().__init__()
    self.mom = mom
    self.eps = eps
    self.mult = nn.parameter(tensor(1.))
    self.add = nn.parameter(tensor(0.))
    self.register_buffer('vars',  torch.ones(1,nf,1,1))
    self.register_buffer('means', torch.zeros(1,nf,1,1))

  def update_stats(self, x):
    m = x.mean((0,2,3), keepdim=True) #taking mean over NHW dimensions
    v = x.var((0,2,3), keepdim=True)
    self.means.lerp_(m, self.mom)
    self.vars.lerp_(v, self.mom)
    return m, v

  def forward(self, x):
    if self.training:
        with torch.no_grad(): m,v = self.update_stats(x)
    else: m,v = self.means,self.vars
    x = (x-m) / (v+self.eps).sqrt()
    return x*self.mults + self.adds